# Machine Learning Test 1


**Exam duration:**
24 hours  

**Required output:**
The uploaded solution must be an ipython notebook (a file of type `ipynb`). All code must be written in python. All text must be written in Markdown.

You may use Colab, Jupyter lab, Jupyter notebook or any other program to complete the exam.

Upload your solution to Github before the deadline. You will get the upload link at bottom of the test.

#### Before submission

Make sure you are submitting error free code!

Click Run --> Restart Kernal and Run all cells.

#### Honor code

As a student of AI Adventures, I `Shruti Karmarkar` pledge to:

1. Uphold the values of academic integrity, honesty, and respect in all of my academic work.

2. Not engage in any form of academic misconduct, including cheating, plagiarism, or using unauthorized resources during exams.

3. Commit to not using ChatGPT or any other AI language model during exam hours, and to rely only on my own knowledge, skills, and resources.

4. Follow all exam rules and instructions, and not bring any unauthorized materials, devices, or resources into the exam room.

5. Report any incidents of academic misconduct that I witness or become aware of, and cooperate fully with any investigation into such incidents.

6. Not ask mentors questions that relate directly to coding during exams, such as "why does my code snippet not work as expected?" or "how do I implement...?"

7. If I am stuck with implementation during an exam, I am allowed to use Google as a resource.

By signing this pledge, I commit to upholding the highest standards of academic integrity and to avoiding all forms of cheating or academic misconduct during my studies at AI Adventures. I understand that violating this pledge may result in disciplinary actions.


# GOOD LUCK
-------------------------------------------------------------------------------------------

## Questions


In this question you will write Python code for processing, analyzing and understanding the social network **Reddit** (www.reddit.com). Reddit is a platform that allows users to upload posts and comment on them, and is divided in _subreddits_, often covering specific themes or areas of interest (for example, [world news](https://www.reddit.com/r/worldnews/), [ukpolitics](https://www.reddit.com/r/ukpolitics/) or [nintendo](https://www.reddit.com/r/nintendo)). You are provided with a subset of Reddit with posts from Covid-related subreddits (e.g., _CoronavirusUK_ or _NoNewNormal_), as well as randomly selected subreddits (e.g., _donaldtrump_ or _razer_).

The `csv` dataset you are provided contains one row per post, and has information about three entities: **posts**, **users** and **subreddits**. The column names are self-explanatory: columns starting with the prefix `user_` describe users, those starting with the prefix `subr_` describe subreddits, the `subreddit` column is the subreddit name, and the rest of the columns are post attributes (`author`, `posted_at`, `title` and post text - the `selftext` column-, number of comments - `num_comments`, `score`, etc.).

In this exercise, you are asked to perform a number of operations to gain insights from the data.

In [ ]:
# suggested imports
import pandas as pd
from nltk.tag import pos_tag
import re
from collections import defaultdict,Counter
from nltk.stem import WordNetLemmatizer
from datetime import datetime
from tqdm import tqdm
import numpy as np
import os
tqdm.pandas()
from ast import literal_eval
# nltk imports, note that these outputs may be different if you are using colab or local jupyter notebooks
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
from urllib import request
import pandas as pd
module_url = f"https://media.githubusercontent.com/media/Ai-Adventures/Test_Datasets/main/ML_Test_1.csv"
module_name = 'ML_Test_1.csv'
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))


df = pd.read_csv('ML_Test_1.csv')
# this fills empty cells with empty strings
df = df.fillna('')

Fetching https://media.githubusercontent.com/media/Ai-Adventures/Test_Datasets/main/ML_Test_1.csv


In [ ]:
df.head()

,author,posted_at,num_comments,score,selftext,subr_created_at,subr_description,subr_faved_by,subr_numb_members,subr_numb_posts,subreddit,title,total_awards_received,upvote_ratio,user_num_posts,user_registered_at,user_upvote_ratio
0,-Howitzer-,2020-08-17 20:26:04,19,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,BREAKING: Trump to begin hiding in mailboxes t...,0,1.00,4661,2012-11-09,-0.658599
1,-Howitzer-,2020-07-06 17:01:48,1,3,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Joe Biden's America,0,0.67,4661,2012-11-09,-0.658599
2,-Howitzer-,2020-09-09 02:29:02,3,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,4 more years and we can erase his legacy for g...,0,1.00,4661,2012-11-09,-0.658599
3,-Howitzer-,2020-06-23 23:02:39,2,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Revelation 9:6 [Transhumanism: The New Religio...,0,1.00,4661,2012-11-09,-0.658599
4,-Howitzer-,2020-08-07 04:13:53,32,622,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,"LOOK HERE, FAT",0,0.88,4661,2012-11-09,-0.658599


## P1.1 - Text data processing

### P1.1.1 - Faved by as lists

The column `subr_faved_by` contains an array of values (names of redditors who added the subreddit to which the current post was submitted), but unfortunately they are in text format, and you would not be able to process them properly without converting them to a suitable python type. You must convert these string values to Python lists, going from

```python
'["user1", "user2" ... ]'
```

to

```python
["user1", "user2" ... ]
```

**What to implement:** Implement a function `transform_faves(df)` which takes as input the original dataframe and returns the same dataframe, but with one additional column called `subr_faved_by_as_list`, where you have the same information as in `subr_faved_by`, but as a python list instead of a string.

In [ ]:
import re
def transform_faves(row):
 return re.findall("'([A-z]+)'", row)


df['subr_faved_by_as_list'] = df.apply(lambda row: transform_faves(row['subr_faved_by']), axis=1)
print(df['subr_faved_by_as_list'])

0        [vergil_never_cry, Jelegend, pianoyeah, salomo...
1        [vergil_never_cry, Jelegend, pianoyeah, salomo...
2        [vergil_never_cry, Jelegend, pianoyeah, salomo...
3        [vergil_never_cry, Jelegend, pianoyeah, salomo...
4        [vergil_never_cry, Jelegend, pianoyeah, salomo...
                               ...                        
19935    [HibikiSS, klondipedia, DonnieMostDefinitely, ...
19936    [HibikiSS, klondipedia, DonnieMostDefinitely, ...
19937    [HibikiSS, klondipedia, DonnieMostDefinitely, ...
19938    [HibikiSS, klondipedia, DonnieMostDefinitely, ...
19939    [HibikiSS, klondipedia, DonnieMostDefinitely, ...
Name: subr_faved_by_as_list, Length: 19940, dtype: object


### P1.1.2 - Merge titles and text bodies

All Reddit posts need to have a title, but a text body is optional. However, we want to be able to access all free text information for each post without having to look at two columns every time.

**What to implement**: A function `concat(df)` that will take as input the original dataframe and will return it with an additional column called `full_text`, which will concatenate `title` and `selftext` columns, but with the following restrictions:

- 1) Wrap the title between `<title>` and `</title>` tags.
- 2) Add a new line (`\n`) between title and selftext, but only in cases where you have both values (see instruction 4).
- 3) Wrap the selftext between `<selftext>` and `</selftext>`.
- 4) You **must not** include the tags in points (1) or (3) if the values for these columns is missing. We will consider a missing value either an empty value (empty string) or a string of only one character (e.g., an emoji). Also, the value of a `full_text` column must not end in the new line character.

In [ ]:
def concat(df):
    df['title_temp'] = df.apply(lambda row: f"<title>{row['title']}</title>", axis=1)
    df['selftext_temp'] = df.apply(lambda row: f"\n<selftext>{row['selftext']}</selftext>" if row['selftext'] else "", axis=1)

    df['full_text'] = df[["title_temp", "selftext_temp"]].apply("".join, axis=1)
    df.drop(["title_temp", "selftext_temp"], axis=1, inplace=True)
    return df

df = concat(df)
df.head(5)

,author,posted_at,num_comments,score,selftext,subr_created_at,subr_description,subr_faved_by,subr_numb_members,subr_numb_posts,subreddit,title,total_awards_received,upvote_ratio,user_num_posts,user_registered_at,user_upvote_ratio,subr_faved_by_as_list,full_text
0,-Howitzer-,2020-08-17 20:26:04,19,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,BREAKING: Trump to begin hiding in mailboxes t...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>BREAKING: Trump to begin hiding in mail...
1,-Howitzer-,2020-07-06 17:01:48,1,3,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Joe Biden's America,0,0.67,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>Joe Biden's America</title>
2,-Howitzer-,2020-09-09 02:29:02,3,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,4 more years and we can erase his legacy for g...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>4 more years and we can erase his legac...
3,-Howitzer-,2020-06-23 23:02:39,2,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,Revelation 9:6 [Transhumanism: The New Religio...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>Revelation 9:6 [Transhumanism: The New ...
4,-Howitzer-,2020-08-07 04:13:53,32,622,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,donaldtrump,"LOOK HERE, FAT",0,0.88,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...","<title>LOOK HERE, FAT</title>"


### P1.1.3 - Enrich posts

We would like to augment our text data with linguistic information. To this end, we will _tokenize_, apply _part-of-speech tagging_, and then we will _lower case_ all the posts.

**What to implement**: A function `enrich_posts(df)` that will take as input the original dataframe and will return it with **two** additional columns: `enriched_title` and `enriched_selftext`. These columns will contain tokenized, pos-tagged and lower cased versions of the original text. **You must implement them in this order**, because the pos tagger uses casing information.

In [ ]:
import spacy
nlp = spacy.load('en_core_web_sm')

def enriched_title(row):
  modified = []
  doc = nlp(row)
  for word in doc:
    modified.append((word.text.lower(), word.pos_))

  return modified

def enriched_selftext(row):
  modified = []
  doc = nlp(row)
  for word in doc:
    modified.append((word.text.lower(), word.pos_))

  return modified


def enrich_posts(df, nlp):
    df['enriched_title'] = df.apply(lambda row: enriched_title(row['title']), axis=1)
    df['enriched_selftext'] = df.apply(lambda row: enriched_selftext(row['selftext']), axis=1)

    return df

df = enrich_posts(df, nlp)

In [ ]:
df.head()

,author,posted_at,num_comments,score,selftext,subr_created_at,subr_description,subr_faved_by,subr_numb_members,subr_numb_posts,...,title,total_awards_received,upvote_ratio,user_num_posts,user_registered_at,user_upvote_ratio,subr_faved_by_as_list,full_text,enriched_title,enriched_selftext
0,-Howitzer-,2020-08-17 20:26:04,19,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,...,BREAKING: Trump to begin hiding in mailboxes t...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>BREAKING: Trump to begin hiding in mail...,"[(breaking, NOUN), (:, PUNCT), (trump, NOUN), ...",[]
1,-Howitzer-,2020-07-06 17:01:48,1,3,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,...,Joe Biden's America,0,0.67,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>Joe Biden's America</title>,"[(joe, PROPN), (biden, PROPN), ('s, PART), (am...",[]
2,-Howitzer-,2020-09-09 02:29:02,3,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,...,4 more years and we can erase his legacy for g...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>4 more years and we can erase his legac...,"[(4, NUM), (more, ADJ), (years, NOUN), (and, C...",[]
3,-Howitzer-,2020-06-23 23:02:39,2,1,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,...,Revelation 9:6 [Transhumanism: The New Religio...,0,1.00,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...",<title>Revelation 9:6 [Transhumanism: The New ...,"[(revelation, NOUN), (9:6, NUM), ([, X), (tran...",[]
4,-Howitzer-,2020-08-07 04:13:53,32,622,,2009-04-29,Subreddit about Donald Trump,"['vergil_never_cry', 'Jelegend', 'pianoyeah', ...",30053,796986,...,"LOOK HERE, FAT",0,0.88,4661,2012-11-09,-0.658599,"[vergil_never_cry, Jelegend, pianoyeah, salomo...","<title>LOOK HERE, FAT</title>","[(look, VERB), (here, ADV), (,, PUNCT), (fat, ...",[]


In [ ]:
df["enriched_title"][0]

[('breaking', 'NOUN'),
 (':', 'PUNCT'),
 ('trump', 'NOUN'),
 ('to', 'PART'),
 ('begin', 'VERB'),
 ('hiding', 'VERB'),
 ('in', 'ADP'),
 ('mailboxes', 'NOUN'),
 ('to', 'PART'),
 ('destroy', 'VERB'),
 ('mail', 'NOUN'),
 ('-', 'PUNCT'),
 ('in', 'ADP'),
 ('ballots', 'NOUN'),
 ('.', 'PUNCT')]

## P1.2 - Answering questions with pandas

In this question, your task is to use pandas to answer questions about the data.

### P1.2.1 - Users with best scores

- Find the users with the highest aggregate scores (over all their posts) for the whole dataset. You should restrict your results to only those whose aggregated score is above 10,000 points, in descending order. Your code should generate a dictionary of the form `{author:aggregated_scores ... }`.

In [ ]:
def user_best_score(df):
    dict1 = {}
    author_grp = df.groupby("author")['score'].sum()
    for i, score in enumerate(author_grp):
      if score > 10000:
       dict1[author_grp.index[i]] = score

    modified_dict = sorted(dict1.items(), key= lambda x: x[1], reverse = True)
    return dict(modified_dict)

print(user_best_score(df))

{'DaFunkJunkie': 250375, 'None': 218846, 'SUPERGUESSOUS': 211611, 'jigsawmap': 210824, 'chrisdh79': 143538, 'hildebrand_rarity': 122464, 'iSlingShlong': 118595, 'hilltopye': 81245, 'tefunka': 79560, 'OldFashionedJizz': 64398, 'JLBesq1981': 58235, 'rspix000': 57107, 'Wagamaga': 47989, 'stem12345679': 47455, 'TheJeck': 26058, 'TheGamerDanYT': 25357, 'TrumpSharted': 21154, 'NotsoPG': 18518, 'SonictheManhog': 18116, 'BlanketMage': 13677, 'NewAltWhoThis': 12771, 'kevinmrr': 11900, 'Dajakesta0624': 11613, 'apocalypticalley': 10382}


### P1.2.2 - Awarded posts

Find the number of posts that have received at least one award. Your query should return only one value.

In [ ]:
df[df['total_awards_received'] >= 1].max()['user_num_posts']

9977

In [ ]:
df.loc[df['total_awards_received'] >= 1]['user_num_posts']

67       1105
69       1105
818      8292
866      6102
911      6900
         ... 
17041    2476
17361    4838
18599    3862
18823    4604
19639     355
Name: user_num_posts, Length: 119, dtype: int64

In [ ]:
def Awarded_posts(df):
    data = df.loc[df['total_awards_received'] >= 1]
    return len(data)

Awarded_posts(df)

119

### P1.2.3 Find Covid

Find the name and description of all subreddits where the name starts with `Covid` or `Corona` and the description contains `covid` or `Covid` anywhere. Your code should generate a dictionary of the form#

```python
  {'Coronavirus':'Place to discuss all things COVID-related',
  ...
  }
```

In [ ]:
dict2 = {}
for name, discription in zip(df['subreddit'], df['subr_description']):
  if (name.startswith("Covid") or name.startswith("Corona")) and ("covid" or "Covid" in discription):
    dict2[name] = discription

dict2

{'Coronavirus': 'Place to discuss all things COVID-related',
 'CoronavirusCA': 'Tracking the Coronavirus/Covid-19 outbreak in California',
 'CoronavirusUS': 'USA/Canada specific information on the coronavirus (SARS-CoV-2) that causes coronavirus disease 2019 (COVID-19)',
 'CoronavirusUK': 'Spreading news, advice and media following the UK’s spread of the virus.',
 'CoronavirusCirclejerk': 'Sub for corona circlejerking, memes, and sharing and discussing stupid, ridiculous, and amusing posts and discussion from the various panic-filled and alarmist Coronavirus-related subs.',
 'CoronavirusDownunder': 'This subreddit is a place to share news, information, resources, and support that relate to the novel coronavirus SARS-CoV-2 and the disease it causes called COVID-19. The primary focus of this sub is to actively monitor the situation in Australia, but all posts on international news and other virus-related topics are welcome, to the extent they are beneficial in keeping those in Australia 

### P1.2.4 - Redditors that favorite the most

Find the users that have favorited the largest number of subreddits. You must produce a pandas dataframe with **two** columns, with the following format:

```python
     redditor	    numb_favs
0	user1           7
1	user2           6
2	user3	       5
3	user4           4
...
```

where the first column is a Redditor username and the second column is the number of distinct subreddits he/she has favorited.

In [ ]:
ex = df.explode('subr_faved_by_as_list')
grouped_d = ex.groupby("author")['subr_faved_by_as_list'].nunique().sort_values(ascending=False).reset_index()
grouped_d.rename(columns = {"author": "redditor", "subr_faved_by_as_list": "numb_favs"}, inplace=True)

In [ ]:
grouped_d.head(5)

,redditor,numb_favs
0,Kinglens311,382
1,AutoModerator,227
2,PerfctSmile,220
3,giveawayguy99,204
4,OliverMarkusMalloy,199


## P1.3 Ethics

Imagine you use the insights gained in this assignment to scan social media for covid-related content, and automatically flag it as `conspiracy` or `not conspiracy` (for example, for hiding potentially harmful tweets or facebook posts). Reflect on the impact of exploiting data science for such an application.


Your answer should address the following:
- Identify the action that, in your opinion, is the weakest.
- Then, justify your choice by critically analyzing the three key principles outlined in the Framework, namely transparency, accountability and fairness.
- Finally, you should propose one solution that explicitly addresses one point related to one of these three principles, reflecting on how your solution would improve the data cycle in this particular use case.

Your answer should be between 500 and 700 words.

**Ans:** In order to fully understand the consequences of automatic flagging of social media posts, let us consider a hypothetical app that sorts the posts into conspiracy and not conspiracy. In my opinion since it's a classification model, it can't possibily be 100% accurate on how it classifies the data and the inaccuracies and biases could be the weakest points here.

1. **Transparency:** the underline process of classifying the posts should be open to the uses. We should know why this app decides what's what. Make it simple for uses to know how it works, what rules it follows and why it lables posts as conspiracy.

  **Fix:** Give us clear rules and explanations. Let us in on the process and why our post got the label. No shady business.

2. **Accountability:** Accountabilty is crucial to justify the system's action when it classifies posts incorrectly as conspiracy. Without accountability the system's mistake can cause harm or unfair treatment

  **Fix:** Create a system where we can say, "Hey, you got it wrong!" and real people check it out. Make the app learn from its mistakes.

3. **Fairness:** Fairness is perticularly important while creating such systems to make sure not to voilate users right to speech in any manner. The system should not be biased and assign incorrect labels that can lead to disproportionate targeting of certain communities or perspectives, violating the principle of fairness.

  **Fix:** Regularly check if the app is biased and correct it. Get a bunch of different people involved to make sure it's fair for everyone.

So, let's keep it real and make sure this app is fair, honest, and ready to own up to slip-ups.